In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Import the Data and Scale it

In [16]:
from numpy import genfromtxt
data = genfromtxt('./data/heston_data_closed.csv', delimiter=',')
X_data = data[:,1:7]
Y_data = data[:,7:8]
print(data.shape)
print(X_data[:10,:])
print(Y_data[100:110,:])


(100000, 8)
[[1.42471461 1.82566    0.76654333 0.39353109 0.6        1.        ]
 [1.35062501 1.8841982  0.32362804 0.2359476  0.60808081 1.        ]
 [1.8140028  1.1977391  0.61770393 0.53178194 0.61616162 1.        ]
 [1.80836255 1.1072207  0.17673168 0.36281445 0.62424242 1.        ]
 [1.83604578 1.11321926 0.87742054 0.41520168 0.63232323 1.        ]
 [1.17431843 1.32544217 0.11921654 0.4708298  0.64040404 1.        ]
 [1.13649332 1.0191621  0.88319825 0.15018406 0.64848485 1.        ]
 [1.3924608  1.65770865 0.08094897 0.80498394 0.65656566 1.        ]
 [1.55831602 1.64742476 0.60395378 0.05199707 0.66464646 1.        ]
 [1.55828875 1.35021076 0.70963542 0.00810449 0.67272727 1.        ]]
[[1.2046148 ]
 [1.1989132 ]
 [1.13436001]
 [1.20692535]
 [1.16247695]
 [1.12897205]
 [1.14405732]
 [1.12495447]
 [1.15317789]
 [1.09256516]]


In [17]:
from sklearn.preprocessing import MinMaxScaler

In [18]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_data)
Y_scaled = scaler.fit_transform(Y_data)

### Create Placeholders and Constants

In [19]:
num_inputs = X_data.shape[1]
num_outputs = Y_data.shape[1]
learning_rate = 0.1
num_steps = 500000
batch_size = 10
num_neurons = 50

X = tf.placeholder(tf.float32, [None, num_inputs])
y = tf.placeholder(tf.float32, [None, num_outputs])

In [20]:
#input: X,y,batch_size
#output: X_batch,y_batch: randomly chosen rows from X,y 
def next_batch(X,y,batch_size):
    max_int = X.shape[0]
    batch_ind = np.random.randint(0,max_int,size=batch_size)
    X_batch = X[batch_ind,:]
    y_batch = y[batch_ind,:]
    
    return X_batch,y_batch

In [21]:
from tensorflow.contrib.layers import fully_connected

#Layers
hidden1 = fully_connected(X, num_neurons, activation_fn=tf.nn.elu)
hidden2 = fully_connected(hidden1, num_neurons, activation_fn=tf.nn.elu)
hidden3 = fully_connected(hidden2, num_neurons, activation_fn=tf.nn.elu)
outputs = fully_connected(hidden3, num_outputs, activation_fn=None)

In [22]:
#Loss Function
loss = tf.reduce_mean(tf.sqrt(tf.square(outputs - y)))  # MSE

In [23]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [24]:
init = tf.global_variables_initializer()

In [25]:
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_steps):
        
        X_batch,Y_batch = next_batch(X_scaled,Y_scaled,batch_size)
        sess.run(train,feed_dict={X: X_batch, y: Y_batch})
        
        if iteration % 1000 == 0:
            
            rmse = loss.eval(feed_dict={X: X_batch, y: Y_batch})
            print(iteration, "\tRMSE:", rmse)
    
    saver.save(sess, "./models/heston_nn")

0 	RMSE: 10.175611
1000 	RMSE: 0.016011423
2000 	RMSE: 0.16020371
3000 	RMSE: 0.04317234
4000 	RMSE: 1.6065696
5000 	RMSE: 0.42447025
6000 	RMSE: 1.5749753
7000 	RMSE: 1.5838085
8000 	RMSE: 1.5864403
9000 	RMSE: 0.15801364
10000 	RMSE: 1.6731678
11000 	RMSE: 0.68594295
12000 	RMSE: 0.095710516
13000 	RMSE: 0.701408
14000 	RMSE: 0.35427293
15000 	RMSE: 0.49919337
16000 	RMSE: 0.02170375
17000 	RMSE: 0.63723516
18000 	RMSE: 0.65041846
19000 	RMSE: 0.09597636
20000 	RMSE: 0.65907997
21000 	RMSE: 0.6540333
22000 	RMSE: 0.3242188
23000 	RMSE: 0.34261942
24000 	RMSE: 0.3355145
25000 	RMSE: 1.6938092
26000 	RMSE: 0.7001623
27000 	RMSE: 0.91657084
28000 	RMSE: 0.9150801
29000 	RMSE: 0.9142107
30000 	RMSE: 0.22432888
31000 	RMSE: 1.6601188
32000 	RMSE: 1.6520001
33000 	RMSE: 1.6086771
34000 	RMSE: 1.6360239
35000 	RMSE: 1.608671
36000 	RMSE: 1.6622765
37000 	RMSE: 0.33976498
38000 	RMSE: 0.35930303
39000 	RMSE: 0.34878033
40000 	RMSE: 0.36132175
41000 	RMSE: 1.0008466
42000 	RMSE: 0.5142025
430